In [1]:
import pandas as pd
import numpy as np
import polars as pl

In [2]:
cols = [# RESPONSE VARIABLE
        #'FUNDING_AGENCY_NAME',
        'FUNDING_OFFICE_NAME', # use for main model 
        #'FUNDING_DEPARTMENT_NAME',
    
        # IDENTIFIERS
        'PIID',
        'IDV_PIID', # Indefinite delivery vehicle procurement instrument identifiers
        'FUNDING_DEPARTMENT_ID',
        # Funding Department ID + IDV PIID + PIID concatenated should give unique ID for a contract
        # null IDV PIID + PIID gives the non-IDV contracts
        'CAGE_CODE', # Code for the entity
        # CAGE Code has been used for the entire dataset so can act as a unique entity ID
        # Unique Entity ID
        #'VENDOR_UEI', # these two don't match about half the time 
        #'ULTIMATE_UEI', # check previous years - DUNS before 2022
        'SOLICITATION_ID',

        # OTHER
        'AWARD_FISCAL_YEAR',
        'VENDOR_ADDRESS_ZIP_CODE', #use only first 5 digits, drop NAs, least amount of NAs 
        'TYPE_OF_SET_ASIDE', # Type of Set Aside determined for the contract action 10N in dictionary 
        'DATE_SIGNED', # The date that a mutually binding agreement was reached. "2022-09-15"
        'EVALUATED_PREFERENCE', #------------
        # Evaluated Preference makes all non-HUBZone entities’ bid prices up 10% when evaluating
    
        # Number of Offers
        'NUMBER_OF_OFFERS_RECEIVED', #a lot of NaNs
        # The number of actual offers/bids received in response to the solicitation.
        'IDV_NUMBER_OF_OFFERS',
        'AWARD_OR_IDV', # award or IDV - mostly Award 


        
        # FILTERING
        'CO_BUS_SIZE_DETERMINATION', # =="SMALL BUSSINES" / "OTHER THAN SMALL BUSINESS"
        'VENDOR_ADDRESS_COUNTRY_NAME', # == USA filter 
        'MODIFICATION_NUMBER', # filtering == 0 
        'EXTENT_COMPETED', # see notes, possibly filter by A,D,E,CDO

        # CONTRACT VALUE
        #'ULTIMATE_CONTRACT_VALUE', # Mutually agreed upon TOTAL contract value including all options 
        #(also includes estimated value of all potential orders for IDVs)
        'DOLLARS_OBLIGATED',

        

    
        # NAICS
        'PRINCIPAL_NAICS_CODE', # 'number' (NAICS codes are updated every 5 years(2012,2017,2022), incosistencies? 
        #The first two digits of the code designate the sector, the third digit designates the subsector, 
        #the fourth digit designates the industry group, the fifth digit designates the NAICS industry, 
        #and the sixth digit designates the national industry. 


        ## GENERAL BUSINESS QUESTIONS
        'CORP_ENTITY_NOT_TAX_EXEMPT', #YES/NO, no nan
        'CORP_ENTITY_TAX_EXEMPT', #YES/NO, no nan 
        'LIMITED_LIABILITY_CORPORATION', #YES/NO, no nan
        'PARTNERSHIP_OR_LLP', #YES/NO, no nan
        'SOLE_PROPREITORSHIP', #YES/NO, no nan
        'SMALL_AGRICULTURAL_COOPERATIVE', #YES/NO, no nan
        'INTERNATIONAL_ORGANIZATION',  #YES/NO, no nan
        'ARCHITECTURE_AND_ENGINEERING', #YES/NO, no nan
        'COMMUNITY_CORP_OWNED_FIRM', #YES/NO, no nan
        'CONSTRUCTION_FIRM', #YES/NO, no nan
        'DOMESTIC_SHELTER', #YES/NO, no nan
        'FOUNDATION', #YES/NO, no nan
        'RESEARCH_AND_DEVELOPMENT', #YES/NO, no nan
        'VETERINARY_HOSPITAL', #YES/NO, no nan
        'HISPANIC_SERVICING_INSTITUTION', #YES/NO, no nan
        'FOR_PROFIT_ORGANIZATION', #YES/NO, no nan
        'EDUCATIONAL_INSTITUTION_FLAG', #YES/NO, no nan
        'MANUFACTURER_OF_GOODS', #YES/NO, no nan
        'SERVICE_PROVIDER', #YES/NO, no nan
        'INDIAN_TRIBE', #YES/NO, no nan # Buy Indian

        # MANUFACTURE - is the product you're selling made in the US 
        # YES, NO, Not a manufactured end product 
        # 'PLACE_OF_MANUFACTURE_CLASS'
        'PLACE_OF_MANUFACTURE', 
        

        # NON-CERTIFIED 
        #Veteran-Owned Business
        'VETERAN_OWNED_FLAG',
        #Service-Disabled Veteran-Owned Business
        'SRDVOB_FLAG',
        # Women-Owned Business
        'WOMEN_OWNED_FLAG']

In [3]:
SAM = pd.read_csv('SAM.csv')

### 2019

In [4]:
test_2019 = pd.read_parquet('2019.parquet', engine='pyarrow',columns=cols)

In [5]:
test_2019 = test_2019[test_2019['CO_BUS_SIZE_DETERMINATION'] == "SMALL BUSINESS"]
test_2019 = test_2019[test_2019['VENDOR_ADDRESS_COUNTRY_NAME'] == "UNITED STATES"]
test_2019 = test_2019[test_2019['EXTENT_COMPETED'].isin(["A", "D", "E", "CDO"])]
test_2019['FUNDING_DEPARTMENT_ID'] = test_2019['FUNDING_DEPARTMENT_ID'].str.strip()
test_2019['IDV_PIID'] = test_2019['IDV_PIID'].str.strip()
test_2019['PIID'] = test_2019['PIID'].str.strip()
test_2019['DOLLARS_OBLIGATED'] = pd.to_numeric(test_2019['DOLLARS_OBLIGATED'], errors='coerce')
m_2019 = pd.merge(test_2019, SAM, on="CAGE_CODE", how="inner")
idx = m_2019.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax()
m_2019 = m_2019.loc[idx]
m_2019 = m_2019[m_2019['DOLLARS_OBLIGATED'] > 0]

In [6]:
m_2019.shape

(35241, 58)

### 2020

In [7]:
test_2020 = pd.read_parquet('2020.parquet', engine='pyarrow',columns=cols)

In [8]:
test_2020 = test_2020[test_2020['CO_BUS_SIZE_DETERMINATION'] == "SMALL BUSINESS"]
test_2020 = test_2020[test_2020['VENDOR_ADDRESS_COUNTRY_NAME'] == "UNITED STATES"]
test_2020 = test_2020[test_2020['EXTENT_COMPETED'].isin(["A", "D", "E", "CDO"])]
test_2020['FUNDING_DEPARTMENT_ID'] = test_2020['FUNDING_DEPARTMENT_ID'].str.strip()
test_2020['IDV_PIID'] = test_2020['IDV_PIID'].str.strip()
test_2020['PIID'] = test_2020['PIID'].str.strip()
test_2020['DOLLARS_OBLIGATED'] = pd.to_numeric(test_2020['DOLLARS_OBLIGATED'], errors='coerce')
m_2020 = pd.merge(test_2020, SAM, on="CAGE_CODE", how="inner")
idx = m_2020.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax()
m_2020 = m_2020.loc[idx]
m_2020 = m_2020[m_2020['DOLLARS_OBLIGATED'] > 0]

In [9]:
m_2020.shape

(36674, 58)

### 2021

In [10]:
test_2021 = pd.read_parquet('2021.parquet', engine='pyarrow',columns=cols)

In [11]:
test_2021 = test_2021[test_2021['CO_BUS_SIZE_DETERMINATION'] == "SMALL BUSINESS"]
test_2021 = test_2021[test_2021['VENDOR_ADDRESS_COUNTRY_NAME'] == "UNITED STATES"]
test_2021 = test_2021[test_2021['EXTENT_COMPETED'].isin(["A", "D", "E", "CDO"])]
test_2021['FUNDING_DEPARTMENT_ID'] = test_2021['FUNDING_DEPARTMENT_ID'].str.strip()
test_2021['IDV_PIID'] = test_2021['IDV_PIID'].str.strip()
test_2021['PIID'] = test_2021['PIID'].str.strip()
test_2021['DOLLARS_OBLIGATED'] = pd.to_numeric(test_2021['DOLLARS_OBLIGATED'], errors='coerce')
m_2021 = pd.merge(test_2021, SAM, on="CAGE_CODE", how="inner")
idx = m_2021.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax()
m_2021 = m_2021.loc[idx]
m_2021 = m_2021[m_2021['DOLLARS_OBLIGATED'] > 0]

In [12]:
m_2021.shape

(38526, 58)

### 2022

In [13]:
test_2022 = pd.read_parquet('2022.parquet', engine='pyarrow',columns=cols)

In [14]:
test_2022 = test_2022[test_2022['CO_BUS_SIZE_DETERMINATION'] == "SMALL BUSINESS"]
test_2022 = test_2022[test_2022['VENDOR_ADDRESS_COUNTRY_NAME'] == "UNITED STATES"]
test_2022 = test_2022[test_2022['EXTENT_COMPETED'].isin(["A", "D", "E", "CDO"])]
test_2022['FUNDING_DEPARTMENT_ID'] = test_2022['FUNDING_DEPARTMENT_ID'].str.strip()
test_2022['IDV_PIID'] = test_2022['IDV_PIID'].str.strip()
test_2022['PIID'] = test_2022['PIID'].str.strip()
test_2022['DOLLARS_OBLIGATED'] = pd.to_numeric(test_2022['DOLLARS_OBLIGATED'], errors='coerce')
m_2022 = pd.merge(test_2022, SAM, on="CAGE_CODE", how="inner")
idx = m_2022.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax()
m_2022 = m_2022.loc[idx]
m_2022 = m_2022[m_2022['DOLLARS_OBLIGATED'] > 0]

In [15]:
m_2022.shape

(38767, 58)

### 2023

In [16]:
test_2023 = pd.read_parquet('FY23_SC_prelim.parquet', engine='pyarrow',columns=cols)

In [17]:
test_2023 = test_2023[test_2023['CO_BUS_SIZE_DETERMINATION'] == "SMALL BUSINESS"]
test_2023 = test_2023[test_2023['VENDOR_ADDRESS_COUNTRY_NAME'] == "UNITED STATES"]
test_2023 = test_2023[test_2023['EXTENT_COMPETED'].isin(["A", "D", "E", "CDO"])]
test_2023['FUNDING_DEPARTMENT_ID'] = test_2023['FUNDING_DEPARTMENT_ID'].str.strip()
test_2023['IDV_PIID'] = test_2023['IDV_PIID'].str.strip()
test_2023['PIID'] = test_2023['PIID'].str.strip()
test_2023['DOLLARS_OBLIGATED'] = pd.to_numeric(test_2023['DOLLARS_OBLIGATED'], errors='coerce')
m_2023 = pd.merge(test_2023, SAM, on="CAGE_CODE", how="inner")
idx = m_2023.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax()
m_2023 = m_2023.loc[idx]
m_2023 = m_2023[m_2023['DOLLARS_OBLIGATED'] > 0]

In [18]:
m_2023.shape

(41382, 58)

### MERGE EVERYTHING

In [19]:
merged_df = pd.concat([m_2019, m_2020, m_2021, m_2022, m_2023], ignore_index=True)

In [20]:
merged_df.shape

(190590, 58)

In [35]:
merged_df.columns

Index(['FUNDING_OFFICE_NAME', 'PIID', 'IDV_PIID', 'FUNDING_DEPARTMENT_ID',
       'CAGE_CODE', 'SOLICITATION_ID', 'AWARD_FISCAL_YEAR',
       'VENDOR_ADDRESS_ZIP_CODE', 'TYPE_OF_SET_ASIDE', 'DATE_SIGNED',
       'EVALUATED_PREFERENCE', 'NUMBER_OF_OFFERS_RECEIVED',
       'IDV_NUMBER_OF_OFFERS', 'AWARD_OR_IDV', 'CO_BUS_SIZE_DETERMINATION',
       'VENDOR_ADDRESS_COUNTRY_NAME', 'MODIFICATION_NUMBER', 'EXTENT_COMPETED',
       'DOLLARS_OBLIGATED', 'PRINCIPAL_NAICS_CODE',
       'CORP_ENTITY_NOT_TAX_EXEMPT', 'CORP_ENTITY_TAX_EXEMPT',
       'LIMITED_LIABILITY_CORPORATION', 'PARTNERSHIP_OR_LLP',
       'SOLE_PROPREITORSHIP', 'SMALL_AGRICULTURAL_COOPERATIVE',
       'INTERNATIONAL_ORGANIZATION', 'ARCHITECTURE_AND_ENGINEERING',
       'COMMUNITY_CORP_OWNED_FIRM', 'CONSTRUCTION_FIRM', 'DOMESTIC_SHELTER',
       'FOUNDATION', 'RESEARCH_AND_DEVELOPMENT', 'VETERINARY_HOSPITAL',
       'HISPANIC_SERVICING_INSTITUTION', 'FOR_PROFIT_ORGANIZATION',
       'EDUCATIONAL_INSTITUTION_FLAG', 'MANUFACTURER

In [21]:
idx = merged_df.groupby(['SOLICITATION_ID','CAGE_CODE'])['DOLLARS_OBLIGATED'].idxmax()
filtered_merged_df = merged_df.loc[idx]

In [23]:
merged_df.shape

(190590, 58)

In [37]:
filtered_merged_df.shape

(123980, 59)

In [24]:
# PLACE_OF_MANUFACTURE
# PLACE_OF_MANUFACTURE_CLASS
# Make NaN into N/A
filtered_merged_df['PLACE_OF_MANUFACTURE'] = filtered_merged_df['PLACE_OF_MANUFACTURE'].fillna('N/A')
# create new  column 
# YES - made in US
# NO - not made in US
# NONE - Not selling a product 

def convert_place_of_manufacture(value):
    if value == 'D':
        return 'YES'
    elif value == 'C':
        return 'NO'
    elif value in ['N/A', 'A', 'G', 'E', 'H', 'L', 'J', 'F', 'K', 'B', 'I']:
        return 'NONE'
    else:
        return 'NONE'

filtered_merged_df['PLACE_OF_MANUFACTURE_CLASS'] = filtered_merged_df['PLACE_OF_MANUFACTURE'].apply(convert_place_of_manufacture)


In [26]:
# zip code was in long format, change to standard 5 digits 
filtered_merged_df['VENDOR_ADDRESS_ZIP_CODE'] = filtered_merged_df['VENDOR_ADDRESS_ZIP_CODE'].astype(str).str[:5]

In [29]:
# assume missing values (not that many) are NONE
filtered_merged_df['TYPE_OF_SET_ASIDE'] = filtered_merged_df['TYPE_OF_SET_ASIDE'].fillna('NONE')

In [31]:
filtered_merged_df['EVALUATED_PREFERENCE'] = filtered_merged_df['EVALUATED_PREFERENCE'].fillna('NONE')

In [33]:
filtered_merged_df = filtered_merged_df.dropna(subset=['CORP_ENTITY_NOT_TAX_EXEMPT'])

In [35]:
filtered_merged_df = filtered_merged_df.dropna(subset=['FUNDING_OFFICE_NAME'])

### FINAL SET FOR MODELING

In [42]:
cols = ['FUNDING_OFFICE_NAME',
        #'CO_BUS_SIZE_DETERMINATION',
        #'VENDOR_ADDRESS_COUNTRY_NAME',
        #'EXTENT_COMPETED',
        #'CAGE_CODE', 
        #'SOLICITATION_ID', 
        #'AWARD_FISCAL_YEAR',
        'VENDOR_ADDRESS_ZIP_CODE', 
        'TYPE_OF_SET_ASIDE', 
        #'DATE_SIGNED',
        'EVALUATED_PREFERENCE', 
        #'NUMBER_OF_OFFERS_RECEIVED', 
        #'DOLLARS_OBLIGATED', 
        'PRINCIPAL_NAICS_CODE',
        'CORP_ENTITY_NOT_TAX_EXEMPT', 
        'LIMITED_LIABILITY_CORPORATION', 
        'PARTNERSHIP_OR_LLP',
        'SOLE_PROPREITORSHIP', 
        'FOR_PROFIT_ORGANIZATION',
        'MANUFACTURER_OF_GOODS',
        'SERVICE_PROVIDER', 
        'INDIAN_TRIBE', 
        'PLACE_OF_MANUFACTURE_CLASS',
        'SELF_8A', 
        'CERT_8A',
        'CERT_HUBZone', 
        'SELF_SDB', 
        'SELF_WOSB', 
        'CERT_WOSB',
        'SELF_EDWOSB', 
        'CERT_EDWOSB', 
        'SELF_VOSB', 
        'SELF_SDVOSB']

In [43]:
final_df = filtered_merged_df[cols]

In [44]:
final_df.shape

(123980, 24)

In [27]:
#pd.set_option('display.float_format', '{:.2f}'.format)

In [45]:
#filtered_merged_df['DOLLARS_OBLIGATED'].describe()

count       123980.00
mean       1043814.28
std        5585954.93
min              0.01
25%          31563.67
50%         139852.18
75%         644211.94
max     1275000000.00
Name: DOLLARS_OBLIGATED, dtype: float64